This notebook is designed to showcase the power of [lvlspy](https://pypi.org/project/lvlspy/) as an API by using it replicate the results from [Gupta and Meyer](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.64.025805). The paper focuses on calculating the equilibration rate between the isomeric state and the ground state of $^{26}\mathrm{Al}$. The methods built into [lvlspy](https://pypi.org/project/lvlspy/) are based on that publication.

To kick things off, we will install any missing packages required for lvlspy and import all the packages required for this notebook. Most are built-in or installed as a dependency for lvlspy.

In [ ]:
#importing system libraries
import sys,subprocess,importlib.util, io, requests

#checking and installing required packages
required  = {'lvlspy','matplotlib','tabulate'}
installed = {pkg for pkg in required if importlib.util.find_spec(pkg) is not None}
missing = required - installed

if missing:
    subprocess.check_call([sys.executable,'-m','pip','install','--quiet',*missing])

#import libraries and modules

import heapq
import tabulate
import numpy as np
import lvlspy.spcoll as lc
import lvlspy.transition as lt
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from lvlspy.io import ensdf,xml
from collections import defaultdict
from IPython.display import HTML, display, Math
from lvlspy.calculate import isomer, evolve, Weisskopf

Now that all the packages are installed and imported, let us download and load into lvlspy an ENSDF file that contains AL26 data. A copy has been uploaded to [OSF](https://osf.io/76qc3) for ease of automation. 

In [ ]:
!curl -o ensdf.026 -JL https://osf.io/76qc3/download #Downloads the ENSDF file containing Al26 data

new_coll = lc.SpColl() #creates a new lvlspy collection
ensdf.update_from_ensdf(new_coll,'ensdf.026','al26') #loads the selected species from the file into the collection

Now let's be sure the file was read and print out the first few levels and their properties

In [ ]:
al26 = new_coll.get()['al26'] #extracts the species from the collection and sets it as its own object

print('Species: ' + al26.get_name())
print('Level Index        Energy (keV)        Multiplicity        ENSDF JPI')
levels = al26.get_levels()

for i in range(6):
    energy = levels[i].get_energy()
    multip = levels[i].get_multiplicity()
    ensdfjp = levels[i].get_properties()['j^pi']
    print(str(i).center(11) + '        ' + str(energy).center(12) + '        ' + str(multip).center(12) + '        ' + ensdfjp.center(9))

Any experienced user of the ENSDF records will tell you how it's incomplete. Specifically how levels have undefined multiplicities. The API will load those levels but assign a property of "useability". If this is set to False, that infers that the level is not useable. For example, let us consider level of index 24 to illustrate:

In [ ]:
print('Energy (keV): ', levels[24].get_energy())
print('Multiplicity: ', levels[24].get_multiplicity())
prop = levels[24].get_properties()
for p in prop:
    print(p,levels[24].get_properties()[p])

As the output shows, the API has assigned a multiplicity by taking the first option in the list. This is due to the fact that to create a levels object, a clear multiplicity is mandatory. If one is left blank in the record, the integer of 1000 is assigned and the 'useability' is set to False.  The 'useability' property is used by the API that something is wrong. This level cannot have clear transitions defined to or from it without some form of averaging, which we will use. But first we must prune the levels that don't have a $J^{\pi}$ defined at all.

In [ ]:
print('number of levels before pruning: ',len(levels))
ensdf.remove_undefined_levels(al26) #remove the levels that have no Jpi defined
levels = al26.get_levels()
print('number of levels after pruning: ',len(levels)) 

Out of the 214 levels read in from the ENSDF record, 185 have at least some form of $J^{\pi}$ defined.
In the Gupta and Meyer paper, they only used 67 levels and modified some of the transitions' reduced matrix coefficients extracted from [Coc et al](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.61.015801). We will trim the loaded the loaded levels and transitions, then update the reduced matrix coefficients for the specific transitions.

In [ ]:
#here we trim the levels from level 67 till the end keeping levels 0 through 66.
#the API will automatically remove the transitions associated with the removed
#levels.

for i in range(67,len(levels)):
    al26.remove_level(levels[i])
levels = al26.get_levels()
print(len(levels))

Some level to level transitions don't exist in the ENSDF record. In the case of $^{26}\mathrm{Al}$, one example is from the isomer state to the ground as indicated by the True boolean below.

In [ ]:
t = al26.get_level_to_level_transition(levels[1],levels[0])
print(t is None)

Since this transition is important to have for such calculations, we will add it and fill in any missing transitions between the remaining levelsmanually. To fill the missing transitions, we can the built in API that will create the missing transiton and calculate an associated Einstein A coefficient based on a Weisskopf esitimate.

In [ ]:
ensdf.fill_missing_ensdf_transitions(al26,26)
t = al26.get_level_to_level_transition(levels[1],levels[0])
print('Upper_level = ', levels.index(t.get_upper_level()), 'to lower level = ', levels.index(t.get_lower_level()), 'with Einstein A coefficient: ', t.get_einstein_a())

Now that one of the important transitions has been created, we'll update some of the transitions. The data from Coc et al. has been listed in the comments in the cell below. If the transtition didn't exist from the record, we will create it first.

In [ ]:
#let's create an iterable list involving the new used from Coc et al.
#2 -> 1 BM3W = 1206
#2 -> 0 BE2W = 36
#3 -> 2 BE2W = 5.8
#3 -> 1 BM1W = 9.7

new_data = [(2,1,'BM3W',1206),(2,0,'BE2W',36),(3,2,'BE2W',5.8),(3,1,'BM1W',2.76)]
for i in new_data:
    t = al26.get_level_to_level_transition(levels[i[0]],levels[i[1]])
    if t is None:
        t = lt.Transition(levels[i[0]],levels[i[1]],0.0)
        e = [levels[i[0]].get_energy(),levels[i[1]].get_energy()]
        j = [(levels[i[0]].get_multiplicity() - 1)//2, (levels[i[1]].get_multiplicity() - 1)//2]
        p = [levels[i[0]].get_properties()['parity'],levels[i[1]].get_properties()['parity']]
        al26.add_transition(t)

    rmc = [(i[2],i[3])]
    ensdf.update_reduced_matrix_coefficient(al26,26,t,rmc)

Now that the data has been fully trimmed and updated as shown from above, let us replicate figure 1 from Gupta and Meyer.

In [ ]:
T = np.logspace(8,10) #setting the temperature array

#initializing the rate arrays
l_10 = np.empty(len(T))
l_01 = np.empty(len(T))

for i,t in enumerate(T):
    l_01[i], l_10[i] = isomer.effective_rate(t,al26)

#Set the fontsize and graph
fontsize = 24

plt.figure(figsize = (8,8))
plt.rcParams['font.size'] = fontsize
plt.xscale('log')
plt.yscale('log')

plt.ylim([1.e-15,1.e+15])
plt.xlim([np.min(T)/1e+9,np.max(T)/1e+9])

plt.plot(T/1e+9,l_10,color = 'red',linewidth = 3,label = 'Isomerization Rate')

plt.ylabel(r'$\mathrm{\lambda_{10}^{eff} [s^{-1}]}$')
plt.xlabel(r'$\mathrm{T_{9}}$')

plt.yticks([1e-15,1e-10,1e-5,1e0,1e+5,1e+10,1e+15])

plt.axhline(y = 0.158,ls = '--',color = 'black',label = r'$0^{+}$ state $\beta^{-}$ Decay Rate')#Al26 beta-decay rate

plt.legend(loc = 'lower right')
plt.show()

The difference between the label is due to an indexing choice. In the paper, the label the ground and isomeric states 1 and 2 respectively. Throughout this notebook, we'll follow the python indexing for simplicity. There is also a missing curve from the graph that's not in the figure that we will address now.

Let's say you want see which levels are affecting the isomerization rate the most. lvlspy allows us to remove and levels and add levels easily. We will do this to test the importance certain transitions have on the entire isomerization rate.

In [ ]:
#re-using the previous graph with the original calculation

plt.figure(figsize = (8,8))
plt.rcParams['font.size'] = fontsize
plt.xscale('log')
plt.yscale('log')

plt.ylim([1.e-15,1.e+15])
plt.xlim([np.min(T)/1e+9,np.max(T)/1e+9])

plt.plot(T/1e+9,l_10,linewidth = 2,label = 'All Transitions')

plt.ylabel(r'$\mathrm{\lambda_{10}^{eff} [s^{-1}]}$')
plt.xlabel(r'$\mathrm{T_{9}}$')

plt.yticks([1e-15,1e-10,1e-5,1e0,1e+5,1e+10,1e+15])

plt.axhline(y = 1.e-1,ls = '--',color = 'black',label = r'$0^{+}$ state $\beta^{-}$ Decay Rate')

#removing transitions, graphing the result, then returning it before removing the next
levels = al26.get_levels()
for i in range(2,6):
    t_remove = al26.get_level_to_level_transition(levels[i],levels[1])
    #removing the transition
    if t_remove is not None:
        al26.remove_transition(t_remove)
        lambda_10_eff = np.empty(len(T))
        lambda_01_eff = np.empty(len(T)) 
        for j,t in enumerate(T):
            lambda_01_eff[j],lambda_10_eff[j] = isomer.effective_rate(t,al26)
        #adding it back after the calculations
        al26.add_transition(t_remove)
        plt.plot(T/1e+9,lambda_10_eff,linewidth = 2,label = 'Transition '+str(i)+ ' to 1 disabled')

plt.legend(loc = 'center left',bbox_to_anchor=(1,0.5))
plt.show()

The transitions that most affect the isomerization rate are 2 $\rightarrow$ 1 and 3 $\rightarrow$ 1. As the graph makes clear, a simple loop can allow any theorist to probe around which transitions are important to the overall effective rate. A powerful tool to help guide both theorists and experimentalists alike.

Now let's analyze the fugacity of the system. Fugacity here deviates away from the pressure definition it uses in thermodynamics, but the term was appropriated due to its description of a state's abundance likeliness to jump to another state with less fugacity. If two states have the same fugacity, they will not exchange abundances. Equilibrium is established when all states have the same fugacity of one.

In [ ]:
#setting up the initial conditions
y0 = np.zeros(len(levels))
y0[0] = 1.0 #setting the total abundance at the ground state

#resetting the temperature
T = 5e+9 #in K

#setting up the time array to be integrated over
time = np.logspace(-15,-10,200)

y,fug = evolve.newton_raphson(al26,T,y0,time) #the method evolves the abundances and fugacities within the same method

eq_prob = al26.compute_equilibrium_probabilities(T) #computes the equilibrium probabilites

We plot the mass fraction evolution of the first 10 levels:

In [ ]:
plt.figure(figsize = (8,8))
plt.rcParams['font.size'] = fontsize
for i in range(10):
    plt.plot(time,y[i,:],label = 'Level ' + str(i))

plt.gca().set_prop_cycle(None)  # Reset the color cycle to align equilibria with network solutions.

for i in range(10):
    plt.plot(time[-1], eq_prob[i], 'x')

plt.xscale('log')

plt.legend(loc = 'center left',bbox_to_anchor=(1,0.5))

plt.show()

We plot their fugacity evolution. The fugacity of a level is the abundance's tendency to leave one level into the other. The higher the fugacity, the more likely you are to leave. Two levels are in equilibrium if they have the same fugacity, and the whole system is said to be in equilibrium when the fugacity of each level is equal to 1.

In [ ]:
#plotting the fugacities

plt.figure(figsize = (8,8))
plt.rcParams['font.size'] = fontsize

for i in range(10):
    plt.plot(time,fug[i,:],label = 'Level '+str(i))

plt.xscale('log')
#plt.ylim([1e-25,1.25])
plt.xlim([min(time), max(time)])
plt.ylabel(r'Fugacity ($\phi$)')
plt.legend(loc = 'center left',bbox_to_anchor=(1,0.5))

plt.show()

To view how all the levels would behave, it would be prudent to animate the fugacity as a function of time

In [ ]:
fig = plt.figure(figsize = (8,8))

#extract energies from the levels in the xml
E = np.empty(len(levels))
for i in range(len(E)):
    E[i] = levels[i].get_energy(units = 'MeV')

def updatefig(i):
    fig.clear()
    fig.suptitle(r'time (s): %8.2e; $T_{9}$ = %4.2f' %(time[i],T/1e+9))
    plt.scatter(fug[0,i],E[0],color = 'green') #ground state
    plt.scatter(fug[1,i],E[1],color = 'red') #isomer
    plt.scatter(fug[2:len(E)-1,i],E[2:len(E)-1],color = 'black')
    plt.axvline(x = fug[0,i],color = 'green')
    plt.axvline(x = fug[1,i],color = 'red')
    plt.xlabel(r'$\phi$')
    plt.ylabel('E (MeV)')
    plt.xscale('log')
    plt.xlim([1e-15,1.5])
    plt.draw()

anim = animation.FuncAnimation(fig,updatefig,len(time))
display(HTML(anim.to_jshtml()))
plt.close() #closes the last frame

All the levels evolve between the fugacities of the ground state and the isomer. As states evolve to the same fugacity, they would then evolve as an ensemble. The system at this temperature reaches equilibrium at around 1.2 $\times\mathrm{10^{-10}}$ s.

The effective rate could be understood via a combinatorial interpretation. The expression $(f_{1}^{in})^{T}(I - F^{T})^{-1}f_{2}^{out}$ from $\lambda_{21}^{eff}$ is actually the effective branching ratio in terms of Graph Theory. The "cascade probability vectors" are defined as $\Gamma_{q}^{in} = (I - F)^{-1}f_{q}^{in}$. The term cascade refers to the de-excitation from higher states into state 'q'. The paper also defines the 'infinite'-arc generalization of $f_{q}^{out}$ with $\Gamma_{q}^{out} = (I - F^{T})^{-1}f_{q}^{out}$. They then go through the proof to rewrite $\lambda_{21}^{eff} = (\lambda_{2}^{out})^{T}\Gamma_{1}^{in}$. Here we sample over temperatures chosen in table 4 from the paper to compare the values of $W_{1}(T)$, $W_{2}(T)$, $G_{1}(T)$, and $G_{2}(T)$, as well as $\lambda_{21}^{eff}$.

In [ ]:
t9 = np.array([0.0100,0.0125,0.0150,0.0175,0.0200,0.0250,0.0300,0.0400,0.0500,0.0600,0.0700,0.0800,0.0900,0.1,
               0.125,0.15,0.175,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.25,1.5,1.75,2.0,2.5,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0])
n = len(t9)

table = np.empty((n,6))

for i in range(n):
    table[i,0],t_9 = t9[i], t9[i]*1e+9
    w_1,w_2,table[i,2],table[i,3],R_1k,R_2k,table[i,4],table[i,5] = isomer.ensemble_weights(t_9,al26)
    l12,table[i,1] = isomer.effective_rate(t_9,al26)

display(Math(r'\;\;\;\;\;\;T_{9} \;\;\;\;\;\;\;\lambda_{21}^{eff} (s^{-1})\;\;\;\;\;  W_{1}(T)\;\;\;\;\;\; W_{2}(T)\;\;\;\;\;\; G_{1}(T)\;\;\;\;\;\;\;G_{2}(T) '))
display(HTML(tabulate.tabulate(table,tablefmt='html',floatfmt=('.4f','.5e','.6f','.6f','.5f','.6f'))))

The table starts deviating from table 4 in [Gupta and Meyer](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.64.025805) at $\mathrm{T_{9}}$ of 0.9. This is due to the the updated ENSDF record used for the higher levels

In [ ]:
class Graph:
    def __init__(self):
        self.adj_list = defaultdict(dict)  # Adjacency list representation

    def add_edge(self, u, v, weight):
        self.adj_list[u][v] = weight  # Directed edge

    def remove_edge(self, u, v):
        if v in self.adj_list[u]:
            del self.adj_list[u][v]

    def dijkstra(self, source, target, limit = 5):
        """Finds the shortest path from source to target using Dijkstra’s algorithm."""
        pq = [(0, source, [])]  # (cost, node, path)
        visited = set()

        while pq:
            cost, node, path = heapq.heappop(pq)

            if path.count(node) > limit:  # Limit how many times a node can be visited
                continue
            visited.add(node)

            path = path + [node]

            if node == target:
                return path, cost

            for neighbor, weight in self.adj_list[node].items():
                if neighbor not in visited:
                    heapq.heappush(pq, (cost + weight, neighbor, path))

        return None, float("inf")  # No path found

    def yen_k_shortest_paths(self, source, target, K=5):
        """Finds the K shortest paths from source to target using Yen's Algorithm."""
        shortest_paths = []
        candidates = []

        # First shortest path using Dijkstra
        first_path, first_cost = self.dijkstra(source, target)
        if not first_path:
            return []

        shortest_paths.append((first_path, first_cost))

        for k in range(1, K):
            for i in range(len(shortest_paths[k - 1][0]) - 1):
                spur_node = shortest_paths[k - 1][0][i]
                root_path = shortest_paths[k - 1][0][:i + 1]

                # Make a temporary copy of the graph
                modified_graph = {node: neighbors.copy() for node, neighbors in self.adj_list.items()}

                # Remove edges that were part of previous paths
                for path, _ in shortest_paths:
                    if path[:i + 1] == root_path and len(path) > i + 1:
                        if path[i + 1] in modified_graph[path[i]]:
                            del modified_graph[path[i]][path[i + 1]]

                # Run Dijkstra from spur node
                temp_graph = Graph()
                temp_graph.adj_list = modified_graph
                spur_path, spur_cost = temp_graph.dijkstra(spur_node, target)

                if spur_path:
                    total_path = root_path[:-1] + spur_path
                    total_cost = sum(self.adj_list[u][v] for u, v in zip(total_path[:-1], total_path[1:])) + spur_cost
                    candidates.append((total_path, total_cost))

            if not candidates:
                break

            candidates.sort(key=lambda x: x[1])
            shortest_paths.append(candidates.pop(0))

        shortest_paths.sort(key = lambda x:x[1])
        return shortest_paths

In [ ]:
graph = Graph()

# Adding bidirectional edges with different weights
transitions = al26.get_transitions()
rate_matrix = al26.compute_rate_matrix(5e+9)

for t in transitions: #fill in the graph edges
    i_upper = levels.index(t.get_upper_level())
    i_lower = levels.index(t.get_lower_level())

    graph.add_edge(i_upper,i_lower,1/(rate_matrix[i_lower,i_upper] + 1e-300))#the 1e-300 is to avoid any 0 issues
    graph.add_edge(i_lower,i_upper,1/(rate_matrix[i_upper,i_lower] + 1e-300))

# Find 5 shortest paths from 1 to 0
k_paths = graph.yen_k_shortest_paths(1, 0, K=5)

# Print results
for i, (path, cost) in enumerate(k_paths):
    print(f"Path {i+1}: {path} with cost {cost}")

In [ ]:
#graphing the 5 shortest paths based on weight
plt.figure(figsize = (10,10))
plt.rcParams['font.size'] = 15

tick_labels = []
tick_values = []

colors = ['black','blue','red','orange','purple']#add more colors if you're looking at more than 5 paths
for j,(path,weight) in enumerate(k_paths):
    plot_legend = []
    n_path = len(path) #total number of transitions
    max_energy = max(path) #index of maximum energy level for transition
    
    for i,en in enumerate(path):
        energy = levels[path[i]].get_energy()
        J = int((levels[path[i]].get_multiplicity() - 1)//2)
        
        #to avoid line and label duplicates in the plot
        if '('+str(energy) + ', '+str(en)+', '+str(J) +')' not in tick_labels and energy not in tick_values:
            
            plt.axhline(y = energy,color = 'black')
            tick_labels.append('('+str(energy) + ', '+str(en)+', '+str(J)+levels[path[i]].get_properties()['parity']+')')
            tick_values.append(energy)
    


    for i in range(n_path-1):
        en_f = levels[path[i+1]].get_energy()
        en_i = levels[path[i]].get_energy()
        delta_E =  en_f - en_i 
        
        if i != n_path-2:
            plot_legend.append(path[i])
            plt.arrow((i+j)/(n_path-1),en_i,1/(n_path - 1),delta_E,color = colors[j],width = 0.0125,length_includes_head = True, 
                    head_length = 100,head_width = 0.1) 
        else:
            plot_legend.append(path[i])
            plot_legend.append(path[i+1])
            plt.arrow((i+j)/(n_path-1),en_i,1/(n_path - 1),delta_E,color = colors[j],width = 0.0125,length_includes_head = True, 
                    head_length = 100,head_width = 0.1,label = 'path: '+str(plot_legend)+' , weight = %3.4e'%(weight)) 

plt.yticks(ticks = tick_values,labels = tick_labels)
plt.tick_params(axis = 'x', which ='both',bottom = False, top = False,labelbottom = False) 
plt.ylabel(r'(E (keV), index, $J^{\pi}$)',ha = 'left', y = 1, rotation = 0,labelpad = 0)
plt.title(r'5 shortest paths at $\mathrm{T_9}$: ' + str(T/1e+9))
plt.legend(bbox_to_anchor=(0.5,0))
plt.tight_layout()
plt.show()